In [1]:
import os
import sys
from pathlib import Path
import jpype

if not jpype.isJVMStarted():
    jpype.startJVM(jpype.getDefaultJVMPath(), "-ea")

jpype.addClassPath("/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/src")
main = jpype.JClass('PythonController')

# Run a sample to test jpype is set up correctly
main.reset()
result = main.step([True, False, False, False, True])
main.close()

0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 99 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
----------------
vx: 0.0, 0.0
 c: 33.0, 2.0
 d: 0
reward: -1.0
done: false
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 99 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0

In [15]:
import numpy as np
obs = np.zeros([1,16, 16])
for i in range(result.observation.length):
    for j in range(result.observation[i].length):
        obs[0][i][j] = result.observation[i][j]

[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 99. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 17.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [1]:
from stable_baselines3.common.env_checker import check_env
from mario_env import MarioEnv

env = MarioEnv()
# It will check your custom environment and output additional warnings if needed
check_env(env)

/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:47: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom feature extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 99 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 17 17 
----------------
vx: 0.0, 0.0
 c: 33.0, 2.0
 d: 0
reward: -1.0
done: false
engine.core.StepResult@45385f75


In [2]:
# Sample 100 random actions in the environment

from mario_env import MarioEnv
import numpy as np


env = MarioEnv()

for i in range(5):
    done = False
    env.reset()
    while not done:
        obs, reward, done, info = env.step(6 + np.random.randint(4)) # sample randomly from right-inputs 
# env.close()

Coords: 3, 13
Coords: 3, 13
Coords: 3, 13
Coords: 3, 13
Coords: 3, 13
Coords: 3, 13
Coords: 4, 13
Coords: 4, 13
Coords: 4, 13
Coords: 4, 13
Coords: 5, 12
Coords: 5, 11
Coords: 5, 10
Coords: 6, 10
Coords: 6, 10
Coords: 7, 9
Coords: 7, 9
Coords: 8, 9
Coords: 8, 10
Coords: 8, 10
Coords: 8, 11
Coords: 8, 11
Coords: 8, 12
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 12
Coords: 8, 11
Coords: 8, 11
Coords: 8, 12
Coords: 8, 12
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 3, 13
Coords: 3, 13
Coords: 3, 13
Coords: 3, 13
Coords: 4, 13
Coords: 4, 13
Coords: 4, 13
Coords: 5, 13
Coords: 5, 13
Coords: 6, 13
Coords: 6, 12
Coords: 6, 11
Coords: 7, 10
Coords: 7, 10
Coords: 7, 10
Coords: 8, 10
Coords: 8, 10
Coords: 8, 11
Coords: 8, 11
Coords: 8, 12
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8, 13
Coords: 8

In [1]:
from stable_baselines3 import DQN, A2C
from mario_vec_env import MarioVecEnv
from mario_env import MarioEnv
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage
from  stable_baselines3.common.vec_env import DummyVecEnv

env = MarioEnv()
print(env.observation_space.shape)
env = DummyVecEnv([lambda: env])
print(env.observation_space.shape)
env = VecFrameStack(env, n_stack=1)
print(env.observation_space.shape)
env = VecTransposeImage(env, skip=True)
print(env.observation_space.shape)
model = A2C('CnnPolicy', env, verbose=1)
# model.learn(total_timesteps=1000)

/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(16, 64, 1)
(16, 64, 1)
(16, 64, 1)
(16, 64, 1)
Using cpu device


RuntimeError: Calculated padded input size per channel: (64 x 1). Kernel size: (8 x 8). Kernel size can't be greater than actual input size

In [7]:
import gym
import torch as th
import torch.nn as nn

from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

from mario_env import MarioEnv
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage
from  stable_baselines3.common.vec_env import DummyVecEnv


class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))


env = MarioEnv(render=False)
print(env.observation_space.shape)
env = DummyVecEnv([lambda: env])
print(env.observation_space.shape)
env = VecFrameStack(env, n_stack=4)
print(env.observation_space.shape)
env = VecTransposeImage(env, skip=True)
print(env.observation_space.shape)

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=128),
)
model = DQN("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1)
model.learn(100000)

(1, 16, 16)
(1, 16, 16)
(4, 16, 16)
(4, 16, 16)
Using cpu device


In [8]:
model.save('cnn_1m_timesteps')